In [2]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import ast
import pickle

In [2]:
df_movies = pd.read_csv('C:\Python\HENRY-Data-Science\Proyecto_1\Datasets\movies_dataset.csv', encoding='utf-8')
df_credits = pd.read_csv('C:\Python\HENRY-Data-Science\Proyecto_1\Datasets\credits.csv', encoding='utf-8')

C:\Users\house\AppData\Local\Temp\ipykernel_3912\204366351.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df_movies = pd.read_csv('C:\Python\HENRY-Data-Science\Proyecto_1\Datasets\movies_dataset.csv', encoding='utf-8')


In [3]:
print(df_movies.shape)
df_movies.head(1)

(45466, 24)


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0


In [4]:
print(df_credits.shape)
df_credits.head(1)

(45476, 3)


,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862


In [5]:
df_movies['id'] = pd.to_numeric(df_movies['id'], errors='coerce')
df_movies.dropna(subset=['id'], inplace=True)
df_movies['id'].astype('int64')

0           862
1          8844
2         15602
3         31357
4         11862
          ...  
45461    439050
45462    111109
45463     67758
45464    227506
45465    461257
Name: id, Length: 45463, dtype: int64

In [6]:
df_joined = df_movies.merge(df_credits, on='id', how='outer')
drop_language = ['en', 'fr', 'it', 'de', 'ar', 'es','uk']
df_joined['popularity'] = pd.to_numeric(df_joined['popularity'], downcast='signed', errors='coerce')
df_joined.query("status == 'Released' and popularity >= 3.0 and release_date >= '1950-1-1' and original_language == @drop_language", inplace=True)
print(df_joined.shape)
df_joined.head(1)

(11419, 26)


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,cast,crew
2,False,NaN,4000000,"[{'id': 80, 'name': 'Crime'}, {'id': 35, 'name...",NaN,5.0,tt0113101,en,Four Rooms,It's Ted the Bellhop's first night on the job....,...,98.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Twelve outrageous guests. Four scandalous requ...,Four Rooms,False,6.5,539.0,"[{'cast_id': 42, 'character': 'Ted the Bellhop...","[{'credit_id': '52fe420dc3a36847f800011b', 'de..."


In [7]:
print(df_joined.shape)
df_joined.columns

(11419, 26)


Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count', 'cast', 'crew'],
      dtype='object')

In [8]:
df_eda = df_joined[['id', 'belongs_to_collection', 'genres', 'overview', 'title', 'cast', 'crew']]
df_eda.dropna(inplace=True)
print(df_eda.shape)
df_eda.head(1)

(2111, 7)


C:\Users\house\AppData\Local\Temp\ipykernel_3912\1100454394.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_eda.dropna(inplace=True)


,id,belongs_to_collection,genres,overview,title,cast,crew
4,11.0,"{'id': 10, 'name': 'Star Wars Collection', 'po...","[{'id': 12, 'name': 'Adventure'}, {'id': 28, '...",Princess Leia is captured and held hostage by ...,Star Wars,"[{'cast_id': 3, 'character': 'Luke Skywalker',...","[{'credit_id': '52fe420dc3a36847f8000437', 'de..."


In [9]:
def extraer(texto):
    lista = []    
    datos = ast.literal_eval(texto)    
    if isinstance(datos, list):        
        for item in datos:
            if isinstance(item, dict) and 'name' in item:
                lista.append(item['name'])    
    elif isinstance(datos, dict) and 'name' in datos:
        lista.append(datos['name'])    
    return lista


def directores(texto):
    lista = []    
    for i in ast.literal_eval(texto):
        if i['job'] == 'Director':
            lista.append(i['name'])
    return lista

def del_espacios(texto):
    lista = []
    for i in texto:
        lista.append(i.replace(" ", ""))
    return lista


In [10]:
df_eda['genres'] = df_eda['genres'].apply(extraer)
df_eda['cast'] = df_eda['cast'].apply(extraer)
df_eda['cast'] = df_eda['cast'].apply(lambda x: x[0:3])
df_eda['belongs_to_collection'] = df_eda['belongs_to_collection'].apply(extraer)
df_eda['crew'] = df_eda['crew'].apply(directores)
df_eda['overview'] = df_eda['overview'].apply(lambda x: x.split())

df_eda['belongs_to_collection'] = df_eda['belongs_to_collection'].apply(del_espacios)
df_eda['genres'] = df_eda['genres'].apply(del_espacios)
df_eda['cast'] = df_eda['cast'].apply(del_espacios)
df_eda['crew'] = df_eda['crew'].apply(del_espacios)

df_eda['overview'] = df_eda['overview'].apply(lambda x: list(filter(lambda y: y and y.strip(), x)))

C:\Users\house\AppData\Local\Temp\ipykernel_3912\2248557828.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_eda['genres'] = df_eda['genres'].apply(extraer)
C:\Users\house\AppData\Local\Temp\ipykernel_3912\2248557828.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_eda['cast'] = df_eda['cast'].apply(extraer)
C:\Users\house\AppData\Local\Temp\ipykernel_3912\2248557828.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_i

In [11]:
df_eda['tags'] = df_eda['belongs_to_collection'] + df_eda['genres'] + df_eda['overview'] + df_eda['cast'] + df_eda['crew']

df_eda.head()

C:\Users\house\AppData\Local\Temp\ipykernel_3912\727673074.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_eda['tags'] = df_eda['belongs_to_collection'] + df_eda['genres'] + df_eda['overview'] + df_eda['cast'] + df_eda['crew']


,id,belongs_to_collection,genres,overview,title,cast,crew,tags
4,11.0,[StarWarsCollection],"[Adventure, Action, ScienceFiction]","[Princess, Leia, is, captured, and, held, host...",Star Wars,"[MarkHamill, HarrisonFord, CarrieFisher]",[GeorgeLucas],"[StarWarsCollection, Adventure, Action, Scienc..."
5,12.0,[FindingNemoCollection],"[Animation, Family]","[Nemo,, an, adventurous, young, clownfish,, is...",Finding Nemo,"[AlbertBrooks, EllenDeGeneres, AlexanderGould]",[AndrewStanton],"[FindingNemoCollection, Animation, Family, Nem..."
9,16.0,[HeartofGoldCollection],"[Drama, Crime, Music]","[Selma,, a, Czech, immigrant, on, the, verge, ...",Dancer in the Dark,"[Björk, CatherineDeneuve, DavidMorse]",[LarsvonTrier],"[HeartofGoldCollection, Drama, Crime, Music, S..."
15,22.0,[PiratesoftheCaribbeanCollection],"[Adventure, Fantasy, Action]","[Jack, Sparrow,, a, freewheeling, 17th-century...",Pirates of the Caribbean: The Curse of the Bla...,"[JohnnyDepp, GeoffreyRush, OrlandoBloom]",[GoreVerbinski],"[PiratesoftheCaribbeanCollection, Adventure, F..."
16,24.0,[KillBillCollection],"[Action, Crime]","[An, assassin, is, shot, at, the, altar, by, h...",Kill Bill: Vol. 1,"[UmaThurman, LucyLiu, VivicaA.Fox]",[QuentinTarantino],"[KillBillCollection, Action, Crime, An, assass..."


In [12]:
df_for_analisys = df_eda.drop(columns=['overview', 'belongs_to_collection', 'genres', 'cast', 'crew'])
df_for_analisys.reset_index(drop=True, inplace=True)


In [13]:
df_for_analisys['tags'] = df_for_analisys['tags'].apply(lambda x: ' '.join(x))
print(df_for_analisys.shape)
df_for_analisys.head()

(2111, 3)


,id,title,tags
0,11.0,Star Wars,StarWarsCollection Adventure Action ScienceFic...
1,12.0,Finding Nemo,"FindingNemoCollection Animation Family Nemo, a..."
2,16.0,Dancer in the Dark,"HeartofGoldCollection Drama Crime Music Selma,..."
3,22.0,Pirates of the Caribbean: The Curse of the Bla...,PiratesoftheCaribbeanCollection Adventure Fant...
4,24.0,Kill Bill: Vol. 1,KillBillCollection Action Crime An assassin is...


In [14]:
print(df_for_analisys.shape)

(2111, 3)


In [15]:
print(df_for_analisys['tags'].iloc[0])

StarWarsCollection Adventure Action ScienceFiction Princess Leia is captured and held hostage by the evil Imperial forces in their effort to take over the galactic Empire. Venturesome Luke Skywalker and dashing captain Han Solo team together with the loveable robot duo R2-D2 and C-3PO to rescue the beautiful princess and restore peace and justice in the Empire. MarkHamill HarrisonFord CarrieFisher GeorgeLucas


In [16]:
df_for_analisys['tags'] = df_for_analisys['tags'].str.replace(r'[.,]', '', regex=True)

In [17]:
df_for_analisys.to_csv('C:\Python\HENRY-Data-Science\Proyecto_1\Datasets\dataset_recomendaciones.csv', index=False)

Creamos el modelo

In [4]:
df_for_analisys = pd.read_csv('C:\Python\HENRY-Data-Science\Proyecto_1\Datasets\dataset_recomendaciones.csv')

SyntaxError: invalid syntax (3891847175.py, line 1)

In [3]:
tv = TfidfVectorizer(max_features=1000, stop_words='english')
vector = tv.fit_transform(df_for_analisys['tags'])
vector.shape


NameError: name 'df_for_analisys' is not defined

In [22]:
similaridad = cosine_similarity(vector)
similaridad

array([[1.        , 0.        , 0.        , ..., 0.01099014, 0.00746715,
        0.02283042],
       [0.        , 1.        , 0.        , ..., 0.        , 0.06953317,
        0.        ],
       [0.        , 0.        , 1.        , ..., 0.        , 0.        ,
        0.06189683],
       ...,
       [0.01099014, 0.        , 0.        , ..., 1.        , 0.        ,
        0.03514016],
       [0.00746715, 0.06953317, 0.        , ..., 0.        , 1.        ,
        0.00569213],
       [0.02283042, 0.        , 0.06189683, ..., 0.03514016, 0.00569213,
        1.        ]])

In [25]:
def recomendacion(pelicula:str):
    indice = df_for_analisys[df_for_analisys['title'] == pelicula].index[0]

    distancia = sorted(list(enumerate(similaridad[indice])), reverse=True, key=lambda x: x[1])
    recomendaciones = []
    for i in distancia[1:6]:
        recomendaciones.append(df_for_analisys.iloc[i[0]].title)
    return recomendaciones

In [26]:
recomendacion('Batman')

['Batman Beyond: Return of the Joker',
 'The Dark Knight Rises',
 'Batman: Bad Blood',
 'Batman Forever',
 'Batman: Mystery of the Batwoman']

In [27]:
pickle.dump(df_for_analisys, open('peliculas.pkl', 'wb'))
pickle.dump(similaridad, open('similaridad.pkl', 'wb'))